<a href="https://www.notion.so/250120_-5c156b8668a8460181fb6ee2e2b3195a">20250120 피드백</a>

# Feedback
- machine 갯수 2개
- insepction_point = mc1과 mc2사이에 고정
    - 단, 이후 insepction point가 바뀔 수 있다는 것을 고려
- 각 machine별 job투입 시점은 맨 처음에 결정된 상태로 job 투입<br><br><br>
- inspect_itv변수이름 변경
    - 마땅히 생각 안나는 관계로 나중에 정하자 

<br><br><br>

package-version은 requirements.txt참고


# 피드백 반영 ENV code

## Packages

In [66]:
import pandas as pd
import numpy as np
import random

## Define Variables

In [67]:
# job
job_num = 5

# insepct
inspect_itv = 2
inspect_proc = 30
inspect_point = 1 ##1번째 Machine을 거친 이후에 inpsect진행

# machine
mc1_range = [10,20]
mc2_range = [20, 30]
machine_range = [mc1_range, mc2_range]
machine_num = len(machine_range)

# machine별 job 투입순서
mc1_job_sequence = []
mc2_job_sequence = []


# Create Job for input

In [68]:
# Create Job 
def make_job_proc_time(machine_num = None, job_num = None, machine_range = None):

    # check parameters
    if machine_num == None:
        raise Exception("machine_num parameter가 없음")
    if job_num == None:
        raise Exception("job_num parameter가 없음")
    if machine_range == None:
        raise Exception("machine range parameter가 없음")
    
    # create mc_job_matrix
    mc_job_matrix = []
    for i in range(machine_num):
        jobs_proc = []
        for j in range(job_num):
            jobs_proc.append(random.randint(machine_range[i][0], machine_range[i][1]))
        mc_job_matrix.append(jobs_proc)
    return mc_job_matrix

mc_job_matrix = make_job_proc_time(
    machine_num=machine_num, job_num=job_num, machine_range=machine_range
)
print(mc_job_matrix)

[[13, 16, 10, 17, 13], [27, 25, 29, 28, 27]]


# Scheduing with ReinforcementLearning Algorithm

- 여기서 Job Scheduing을 진행한다.
- 해당 블록에 RL Moudle이 들어감
- 일단 지금은 각 machine별 job투입 시점을 random하게 정한다. 

In [69]:
# job_seqence 생성 코드
x = 0
job_sequence = []

for _ in range(job_num):
    job_sequence.append(x)
    x+=1




# machine별 job_sequence(job투입순서)
random.shuffle(job_sequence)
mc1_job_sequence = job_sequence[:]  # job_sequence 복사

random.shuffle(job_sequence)
mc2_job_sequence = job_sequence[:]  # job_sequence 복사

print(mc1_job_sequence)
print(mc2_job_sequence)


# inspect 대상 생성 코드 (한개만 한다고 가정)
job_inspection = []
inspection = random.randint(0,5)

for _ in range(job_num):
    if _ == inspection:
        job_inspection.append(1)
    else:
        job_inspection.append(0)

RL_result = {
    "mc1_job_sequence": mc1_job_sequence,
    "mc2_job_sequence": mc2_job_sequence,
    "job_inspection": job_inspection 
}

print(RL_result)

[3, 0, 4, 2, 1]
[3, 2, 4, 0, 1]
{'mc1_job_sequence': [3, 0, 4, 2, 1], 'mc2_job_sequence': [3, 2, 4, 0, 1], 'job_inspection': [0, 0, 1, 0, 0]}


# Create and Fill report

- job의 순서가 바뀌는 시점마다 그 시점까지의 I/O시점 data가 담긴 report를 제출한다. 
- mc1 - inpsection - mc2과정에서는 mc2 input시점에서 job의 순서가 바뀌므로 inspection이 끝난 시점에서 한번의 리포트를 제출하고 이후 mc2과정이 끝나고 나서 최종 리포트를 제출한다.

In [70]:
# 1. Create Report 
# machine1 ~ inspection I/O report

report = pd.DataFrame({
    'job': RL_result.get('mc1_job_sequence'),
    'inspect': RL_result.get('job_inspection')
})


# 계산을 위한 임시 칼럼 생성
report['temp'] = 0

# 기계, 검사 정보 칼럼 생성
col_list = [[f'mc{i}i', f'mc{i}o'] for i in range(machine_num)]
concat = ['insi', 'inso']

# inspection과정 삽입
col_list.insert(inspect_point, concat)
print(col_list)

# list 평면화
col_list = [item for temp in col_list for item in temp]
print(col_list)

# Column 병합
report[col_list] = None
report

[['mc0i', 'mc0o'], ['insi', 'inso'], ['mc1i', 'mc1o']]
['mc0i', 'mc0o', 'insi', 'inso', 'mc1i', 'mc1o']


,job,inspect,temp,mc0i,mc0o,insi,inso,mc1i,mc1o
0,3,0,0,None,None,None,None,None,None
1,0,0,0,None,None,None,None,None,None
2,4,1,0,None,None,None,None,None,None
3,2,0,0,None,None,None,None,None,None
4,1,0,0,None,None,None,None,None,None


# Non-Permutation Process Logic

## Step1. Machine1 - Inspection

In [71]:
# Non-Permutation
print(report)
total_proc_num = machine_num * 2  + 2 
print(f"total proc num = {machine_num * 2 + 2}")

for seq in range(0, job_num):
    for mc in range(3, 3 + inspect_point*2 + 2):
        
        # inspect in column 검터 중인 경우
        if mc == (3 + inspect_point*2):
            ##첫 번째 작업일 경우 예외처리
            if seq == 0:
                report.iloc[0, mc] = report.iloc[0, mc-1]
            # 첫 번째 작업이 아니라면 
            else:
                ##inspect 대상일 경우
                if report.loc[seq, 'inspect'] == 1:
                    previous_insp = -1
                    previous_ins_seq = -1

                    # 이전에 insepction을 거친 job이 있는지 탐색
                    # 있다면 그 job의 inpsection out타임과 지금 job의 이전 machine의 output time을 비교해서 더 큰값을 insi로 집어넣는다.
                    pre_porc = range(0, seq)
                    for i in reversed(range(0, seq)):
                        previous_insp = report.loc[i,'inspect']
                        if (previous_insp == 1):
                            previous_ins_seq = i
                            break
                    # 지금이 처음이라면
                    if (previous_insp == 0):
                        report.iloc[seq, mc] = report.iloc[seq, mc-1]

                    # 이전 단계에 inspection을 거친 job이 있다면 
                    else:
                        report.iloc[seq, mc] = max(report.iloc[seq, mc-1], report.iloc[previous_ins_seq, mc+1])
                # inspection 대상이 아닐경우 
                else:
                    report.iloc[seq, mc] = report.iloc[seq, mc-1]

        # inspect out칼럼 검터 중인 경우
        elif mc == (3 + inspect_point*2 + 1):
            # insepct 대상 O
            if report.loc[seq, 'inspect'] == 1:
                report.iloc[seq, mc] = report.iloc[seq, mc-1] + inspect_proc
            
            # insepct 대상 X
            else:
                report.iloc[seq, mc] = report.iloc[seq, mc-1]
        

        # 일반 기계 in 칼럼 검토 중인 경우
        elif mc % 2 == 1:
            # 첫번째 작업 예외
            if seq == 0:
                report.iloc[0, mc] = report.iloc[0, mc-1]
            else:
                report.iloc[seq, mc] = max(report.iloc[seq, mc-1], report.iloc[seq-1, mc+1])
        
        # 일반 기계 out 칼럼 검토 중인 경우
        else:
            col = report.columns[mc]
            machine = int(col[2:3])
            job = report.iloc[seq, 0]
            report.iloc[seq, mc] = report.iloc[seq, mc-1] + mc_job_matrix[machine][job]


print("-----------------Step1 결과 -----------------\n ")
print(f"mc1_job_input_order : {mc1_job_sequence}\n")
report1 = report
print(report1)
print("---------------------------------- ----------------------------------\n\n")
                    

   job  inspect  temp  mc0i  mc0o  insi  inso  mc1i  mc1o
0    3        0     0  None  None  None  None  None  None
1    0        0     0  None  None  None  None  None  None
2    4        1     0  None  None  None  None  None  None
3    2        0     0  None  None  None  None  None  None
4    1        0     0  None  None  None  None  None  None
total proc num = 6
-----------------Step1 결과 -----------------
 
mc1_job_input_order : [3, 0, 4, 2, 1]

   job  inspect  temp mc0i mc0o insi inso  mc1i  mc1o
0    3        0     0    0   17   17   17  None  None
1    0        0     0   17   30   30   30  None  None
2    4        1     0   30   43   43   73  None  None
3    2        0     0   43   53   53   53  None  None
4    1        0     0   53   69   69   69  None  None
---------------------------------- ----------------------------------




## Step2. Machine2
- inpsect는 단 한번 진행하며, 그 시점은 mc1이 끝난 이후로 고정된 상태라고 가정한 상태에서의 코드이다.
- 즉 insepction과정이 더이상 일어나지 않는 상황을 생각하고 짠 코드이다.
- 후에 앞선 로직에 변화가 있을 경우 (inpsection횟수가 늘어 난다거나 inspection시점이 변한다거나)하면 해당 Logic은 수정이 필요하다.

In [73]:
## Job순서 바꾸는 Logic
# 1. job_num을 Index로 셋팅하고 mc2_job_sequence에 따라 재정렬
report = report.set_index('job').reindex(RL_result.get('mc2_job_sequence'))
# 2. 원래 index를 다시 가져옴 (0,1,2,3,4)
report = report.reset_index(drop=False)

for seq in range(0, job_num):
    for mc in range(3 + inspect_point * 2 + 1  + 1, 3 + total_proc_num):
        
        # 일반 Mahcine Input
        if mc % 2 == 1:
            # 첫번째 작업 예외 처리
            if seq == 0:
                report.iloc[seq, mc] = report.iloc[seq, mc-1]
            else:
                report.iloc[seq, mc] = max(report.iloc[seq, mc-1], report.iloc[seq-1, mc+1])

        # 일반 Machine Output
        else:
            col = report.columns[mc]
            # machine number parsing
            macine = int(col[2:3])
            # job number parsing
            job = report.iloc[seq, 0]
            report.iloc[seq, mc] = report.iloc[seq, mc-1] + mc_job_matrix[machine][job]


print("-----------------Step2 결과 -----------------\n ")
print(f"mc2_job_input_order : {mc2_job_sequence}\n")
report2 = report
print(report2)
print("---------------------------------- ----------------------------------\n\n")

어휴 진짜 씨발 진짜 
   job  inspect  temp mc0i mc0o insi inso mc1i mc1o
0    3        0     0    0   17   17   17   17   34
1    2        0     0   43   53   53   53   53   63
2    4        1     0   30   43   43   73   73   86
3    0        0     0   17   30   30   30   86   99
4    1        0     0   53   69   69   69   99  115
-----------------Step2 결과 -----------------
 
mc2_job_input_order : [3, 2, 4, 0, 1]

   job  inspect  temp mc0i mc0o insi inso mc1i mc1o
0    3        0     0    0   17   17   17   17   34
1    2        0     0   43   53   53   53   53   63
2    4        1     0   30   43   43   73   73   86
3    0        0     0   17   30   30   30   86   99
4    1        0     0   53   69   69   69   99  115
---------------------------------- ----------------------------------




In [75]:
print(report1)
print(report2)

   job  inspect  temp mc0i mc0o insi inso  mc1i  mc1o
0    3        0     0    0   17   17   17  None  None
1    0        0     0   17   30   30   30  None  None
2    4        1     0   30   43   43   73  None  None
3    2        0     0   43   53   53   53  None  None
4    1        0     0   53   69   69   69  None  None
   job  inspect  temp mc0i mc0o insi inso mc1i mc1o
0    3        0     0    0   17   17   17   17   34
1    2        0     0   43   53   53   53   53   63
2    4        1     0   30   43   43   73   73   86
3    0        0     0   17   30   30   30   86   99
4    1        0     0   53   69   69   69   99  115
